In [59]:
# %load template.py
# %load template.py
import glob
import io
import ipyleaflet
import IPython.display
import ipyvolume.pylab as p3
import json
import matplotlib.cm
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pdal
import PIL
import pyproj
import requests
import fiona
import shapely.geometry
from shapely.geometry import Polygon, mapping
import scipy.spatial
import sys
import urllib.request

%load_ext autoreload
%autoreload 2
    
sys.path.append('../src')
from pcl_utils import local_max

# Url for aerial imagery
# IVaerial = "https://geoservices.informatievlaanderen.be/raadpleegdiensten/ogw/wms?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&CRS=EPSG:31370&BBOX={0},{1},{2},{3}&WIDTH=512&HEIGHT=512&LAYERS=OGWRGB13_15VL&STYLES=default&FORMAT=image/png"

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# look at tunnel site
# from bigLaz- crop, 
bigLaz = "/media/thistle/Passport/gis/Idaho_Springs/lidar_downloads/outputs/bigLaz.laz"

In [12]:
# get tunnel_poly
polys = {}
sites_fp = "/home/thistle/Documents/dev/ML/idaho1/data/gis/sites_of_interest/sites.shp"
with fiona.open(sites_fp, "r") as f:
    for each in f:
        polys[each['properties']['name']] =  each['geometry']['coordinates'][0]

In [13]:
polys

{'tunnel': [(459124.6879059952, 4399616.3684700085),
  (459903.65261469834, 4399671.354214152),
  (459919.69012340694, 4399265.834351093),
  (459729.53109157644, 4398945.08417692),
  (459223.2040309194, 4399247.505769711),
  (459124.6879059952, 4399616.3684700085)],
 'gulch': [(461450.12666874146, 4399439.955874214),
  (461766.29469756805, 4399430.791583523),
  (461825.8625870571, 4399100.877118661),
  (461532.6052849571, 4398876.35199674),
  (461367.6480525258, 4398523.526805151),
  (461005.65857024613, 4398069.894415965),
  (460845.2834831602, 4398317.330264612),
  (460982.74784351955, 4398720.559055),
  (461445.5445233962, 4399178.773589531),
  (461450.12666874146, 4399439.955874214)],
 'bridge': [(462448.03200972814, 4399742.950235166),
  (462551.13027999847, 4399477.185805135),
  (462326.60515807645, 4399053.337360691),
  (461979.50764816644, 4399214.857984114),
  (461917.6486860043, 4399379.815216547),
  (462448.03200972814, 4399742.950235166)]}

In [15]:
# looking at tunnel site
tunnel = Polygon(polys['tunnel'])
tunnel.to_wkt()

'POLYGON ((459124.6879059951752424 4399616.3684700084850192, 459903.6526146983378567 4399671.3542141523212194, 459919.6901234069373459 4399265.8343510925769806, 459729.5310915764421225 4398945.0841769203543663, 459223.2040309194126166 4399247.5057697109878063, 459124.6879059951752424 4399616.3684700084850192))'

In [18]:
bigLaz

'/media/thistle/Passport/gis/Idaho_Springs/lidar_downloads/outputs/bigLaz.laz'

In [20]:
output_las = '/media/thistle/Passport/gis/Idaho_Springs/lidar_downloads/outputs/tunnel1.las'
pipe1 = {"pipeline":[
    {
        "type": "readers.las",
        "filename": bigLaz
    },
    {
        "type": "filters.crop",
        "polygon": tunnel.to_wkt()
    },
    {
        "type": "filters.hag"  
    },
    {
        "type": "filters.eigenvalues",
        "knn": 16
    },
    {
        "type": "filters.normal",
        "knn": 16
    },
    {
        "type": "writers.las",
        "filename": output_las
    }

]}

In [21]:
pp = pdal.Pipeline(json.dumps(pipe1))
pp.validate()

True

In [23]:
%%time
numpts = pp.execute()
print(f"{numpts:,} processed")

2,700,540 processed
CPU times: user 1min 53s, sys: 2.12 s, total: 1min 55s
Wall time: 1min 56s


In [41]:
# !dir(pp)

In [26]:
arr = pp.arrays[0]

In [29]:
description = arr.dtype.descr

In [40]:
cols = [each for each, _ in description]
print(cols)

['X', 'Y', 'Z', 'Intensity', 'ReturnNumber', 'NumberOfReturns', 'ScanDirectionFlag', 'EdgeOfFlightLine', 'Classification', 'ScanAngleRank', 'UserData', 'PointSourceId', 'GpsTime', 'Red', 'Green', 'Blue', 'HeightAboveGround', 'Eigenvalue0', 'Eigenvalue1', 'Eigenvalue2', 'NormalX', 'NormalY', 'NormalZ', 'Curvature']


In [38]:
df = pd.DataFrame({col:arr[col] for col in cols})
df.shape

(2700540, 24)

In [39]:
# df.describe()
# df.info()
df.head()

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,...,Green,Blue,HeightAboveGround,Eigenvalue0,Eigenvalue1,Eigenvalue2,NormalX,NormalY,NormalZ,Curvature
0,459155.87,4399499.90,2253.06,51,1,1,0,0,2,-10.0,...,0,0,0.0,0.001872,0.091064,0.195129,-0.051803,-0.094295,0.994196,0.006500
1,459156.33,4399499.51,2253.04,51,1,1,1,0,2,-10.0,...,0,0,0.0,0.001825,0.073161,0.216886,-0.054036,-0.078679,0.995434,0.006251
2,459156.36,4399498.81,2252.98,48,1,1,1,0,2,-10.0,...,0,0,0.0,0.001861,0.064083,0.232987,-0.090483,-0.028608,0.995487,0.006226
3,459156.39,4399498.10,2253.02,46,1,1,1,0,2,-10.0,...,0,0,0.0,0.001429,0.067241,0.213996,-0.061574,-0.054032,0.996639,0.005055
4,459157.26,4399497.38,2252.92,52,1,1,0,0,2,-10.0,...,0,0,0.0,0.001023,0.097466,0.169182,-0.056240,-0.099127,0.993484,0.003821


In [42]:
df['X_0'] = df['X']
df['Y_0'] = df['Y']
df['Z_0'] = df['Z']
df['X'] = df['X'] - df['X_0'].min()
df['Y'] = df['Y'] - df['Y_0'].min()
df['Z'] = df['Z'] - df['Z_0'].min()

In [43]:
df['X_0'].min(), df['X'].min()

(459124.9, 0.0)

In [44]:
df['X_0'].max(), df['X'].max()

(459919.38, 794.4799999999814)

In [50]:
fig = p3.figure(width=1000)
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'
all_points = p3.scatter(df['Y'], df['Z'], df['X'], color='red', size=.2)
p3.squarelim()
p3.show()

# you can see the powerlines well

In [51]:
# Color ground in grey
df['ground'] = df['Classification']==2
ground = p3.scatter(df.loc[df['ground'],'Y'].values, df.loc[df['ground'],'Z'].values, df.loc[df['ground'],'X'].values, color='red', size=.2)
non_ground = p3.scatter(df.loc[~df['ground'],'Y'].values, df.loc[~df['ground'],'Z'].values, df.loc[~df['ground'],'X'].values, color='red', size=.2)
fig.scatters.append(ground)
fig.scatters.append(non_ground)
all_points.visible = False
ground.color='lightgrey'

In [53]:
non_ground.visible = True  # False

In [47]:
df['Classification'].value_counts()

2     919143
18    719503
1     712339
17    348496
7       1036
11        23
Name: Classification, dtype: int64

In [54]:
ground_delaunay = scipy.spatial.Delaunay(df.loc[df['ground'],['X','Y']])
ground_surf = p3.plot_trisurf(df.loc[df['ground'],'Y'], df.loc[df['ground'],'Z'], df.loc[df['ground'],'X'], ground_delaunay.simplices, color='lightgrey')
fig.meshes.append(ground_surf)
# ground_surf.color = "salmon"
ground.visible=False
non_ground.visible=False

In [55]:
# Color points according to flatness
df['flatness'] = df['Eigenvalue0'] 
non_ground.color=matplotlib.cm.viridis(df.loc[~df['ground'],'flatness']*4)[:,0:3]

In [56]:
# Separate between trees and the rest
df['tree_potential'] = (df['Classification']==1) & (df['HeightAboveGround'] >= 2) & (df['flatness'] > .05) &  (df['NumberOfReturns'] - df['ReturnNumber'] >= 1) 
df['other'] = ~df['ground'] & ~df['tree_potential']
tree_potential = p3.scatter(df.loc[df['tree_potential'],'Y'].values, df.loc[df['tree_potential'],'Z'].values, df.loc[df['tree_potential'],'X'].values, color=matplotlib.cm.viridis(df.loc[df['tree_potential'],'flatness']*4)[:,0:3], size=.2)
other = p3.scatter(df.loc[df['other'],'Y'].values, df.loc[df['other'],'Z'].values, df.loc[df['other'],'X'].values, color=matplotlib.cm.viridis(df.loc[df['other'],'flatness']*4)[:,0:3], size=.2)
non_ground.visible=False
tree_potential.color='darkgreen'
other.color='red'

In [57]:
other.visible=False

In [73]:
lep = local_max(df.loc[df['tree_potential'],['X','Y','Z','HeightAboveGround']], radius=3, density_threshold=5)

treetop_spheres = p3.scatter(lep['Y'].values, lep['Z'].values, lep['X'].values, color='red', size=.5, marker='sphere')
fig.scatters.append(treetop_spheres)

In [74]:
treetop_spheres.color = matplotlib.cm.tab20(np.arange(len(lep['Z']))%10)[:,0:3]

In [75]:
kdtree = scipy.spatial.kdtree.KDTree(lep[['X','Y','Z']])
dist, idx = kdtree.query(df.loc[df['tree_potential'],['X','Y','Z']].values)
tree_potential.color=matplotlib.cm.tab20(idx%10)[:,0:3]
df.loc[df['tree_potential'], 'tree_idx'] = idx

In [76]:
medians = df.groupby('tree_idx')[['X','Y','Z']].median()
for axis in ['X','Y','Z']:
    df['d'+axis] = df[axis] - df['tree_idx'].map(medians[axis])
df['radius'] = np.linalg.norm(df[['dX', 'dY', 'dZ']].values, axis=1)
radii = pd.DataFrame([df.groupby('tree_idx')['radius'].quantile(.5), lep['HeightAboveGround'].values*.4]).min()

In [77]:
scale = max(df['X'].max() - df['X'].min(), df['Y'].max() - df['Y'].min())
treetop_spheres.x = medians['Y']
treetop_spheres.y = medians['Z']
treetop_spheres.z = medians['X']
treetop_spheres.size = radii * 100 / scale

In [78]:
# the power line displays quite distinctly

In [72]:
tree_potential.visible = False
other.visible = False

In [71]:
other.visible = True
treetop_spheres.color='darkgreen'
p3.style.use('minimal')

In [ ]:
# Continue working to find tallest trees not including powerlines